<a href="https://colab.research.google.com/github/iued-uni-heidelberg/DAAD-Training-2021/blob/main/cord19download2text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading and reading CORD19 corpus
This notebook downloads and reads the free cord19 corpus into one file. The notebook is hosted at IÜD, Heidelberg University github repository https://github.com/iued-uni-heidelberg/cord19

CORD19 (covid-19) open-source corpus is available from https://www.semanticscholar.org/cord19/download. 

Documentation is available at https://github.com/allenai/cord19

The original files are in json format. The output file is in plain text format; documents are separated (by default) by \<doc id="doc1000001"> ... \</doc> tags

The purpose of the plain text file is for further processing, e.g., generating linguistic annotation using the TreeTagger or the Standford parser for part-of-speech annotation or dependency / constituency parsing.



## Downloading CORD19 corpus

The corpus is downloaded and extracted from https://www.semanticscholar.org/cord19/download

Please check the link above: if you need the latest release of the corpus or if you would like to choose another release. Currently the 2021-08-30 release is downloaded.

File size is ~11GB
expected download time ~5 min


In [1]:
!wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2021-08-30.tar.gz

--2021-09-05 10:29:02--  https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2021-08-30.tar.gz
Resolving ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)... 52.92.128.170
Connecting to ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)|52.92.128.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12069361365 (11G) [binary/octet-stream]
Saving to: ‘cord-19_2021-08-30.tar.gz’

cord-19_2021-08-30. 100%[===================>]  11.24G  34.3MB/s    in 5m 10s  

2021-09-05 10:34:12 (37.2 MB/s) - ‘cord-19_2021-08-30.tar.gz’ saved [12069361365/12069361365]



Extracting cord-19 corpus, approximate time ~ 4 min

In [2]:
!tar -xvzf cord-19_2021-08-30.tar.gz

2021-08-30/changelog
2021-08-30/cord_19_embeddings.tar.gz
2021-08-30/document_parses.tar.gz
2021-08-30/metadata.csv


Removing initial archive to free some disk space

In [3]:
!rm cord-19_2021-08-30.tar.gz

Extracting document parsers, which contain individual articles in separate json files. This is expected to take a long time.

In [ ]:
!tar -xvzf 2021-08-30/document_parses.tar.gz

Streaming output truncated to the last 5000 lines.
document_parses/pmc_json/PMC7818697.xml.json
document_parses/pdf_json/5933803f774f93fe134286714344a8dcc5791487.json
document_parses/pmc_json/PMC7907501.xml.json
document_parses/pdf_json/4b48a21d6971202363d0430c67f4e978d99fb9b4.json
document_parses/pdf_json/245eafd7791411c7efc016d414d5c6653d41ec88.json
document_parses/pmc_json/PMC7885546.xml.json
document_parses/pdf_json/ea6c41912e47a0e415af7c85d63ef4f032395cfc.json
document_parses/pmc_json/PMC7548505.xml.json
document_parses/pdf_json/c54700c3d8a26d78c31a24661710448953e77843.json
document_parses/pmc_json/PMC7149947.xml.json
document_parses/pdf_json/fcb34ce833647afa1a3bf637da234e56a733c9cf.json
document_parses/pmc_json/PMC8061634.xml.json
document_parses/pdf_json/0e8eb9708be598e76fdda76bd8eff3108d0520ef.json
document_parses/pdf_json/5f8e6ee3624a9ff4acecacc63ac7157d21c67631.json
document_parses/pmc_json/PMC7494383.xml.json
document_parses/pdf_json/a364a570aaa5b09ab7678ebdb53c0664c8ae8dbc.

## Reading json directory and merging into text file(s)

This is a class for reading a directory with json files and writing them to a single file or split into several text file, with "split_by_docs=N", N documents in each file.

In [ ]:
# -*- coding: utf-8 -*-
# Python script to open each file, read json input and copy to one text file for subsequent processing
import os, re, sys
import json

class clJsonDir2txt(object):
    '''
    @author Bogdan Babych, IÜD, Heidelberg University
    @email bogdan [dot] babych [at] iued [dot] uni-heidelberg [dot] de
    a script for processing covid-19 corpus:
    @url https://www.semanticscholar.org/cord19 @url https://www.semanticscholar.org/cord19/download
        recursively reads files from a directory, and glues them together into a single corpus file

    @todo:
        working with sections - collect titles of all sections; frequent sections; select argumentative sections (e.g., discussion, analysis...)
        - to compare descriptive and argumentative parts of the corpus

        experimenting with different annotations (pos, parsing... ); MT quality evaluation...
    '''
    def __init__(self, SDirName, output_file = 'corpus_out.txt', include_title = True, include_refs = True, include_authors = True, tag='doc', id=1000000, split_by_docs = 0): # initialising by openning the directories
        self.SOutput_file = output_file
        self.BInclTitle = include_title # implemented
        self.BInclRefs = include_refs # not implemented yet
        self.BInclAuth = include_authors # not implemented yet
        self.STag = tag
        self.ID = id
        self.ISplitByDocs = int(split_by_docs)
        # print(self.ISplitByDocs)
        self.openDir(SDirName)
        return


    def openDir(self, path): # implementation of recursively openning directories from a given rule directory and reading each file recursively into a string
        i = 0
        if self.ISplitByDocs:
            SPartFile = "part1000000" + self.SOutput_file
            FOut = open(SPartFile, 'w')
        else:
            FOut = open(self.SOutput_file, 'w')

        for root,d_names,f_names in os.walk(path):
            for f in f_names:
                i+=1
                if i%1000==0: print(str(i) + '. Processing: ' + f)
                fullpath = os.path.join(root, f)
                # print(fullpath)
                try:
                    FIn = open(fullpath,'r')
                    SIn = FIn.read()
                    SText2Write = self.procFile(SIn,f,i)
                    if SText2Write: FOut.write(SText2Write) # if the string is not empty then write to file
                    FIn.close()
                except:
                    print(f'file {f} cannot be read or processed')

                # splitting output into chunks of "split_by_docs" size
                if self.ISplitByDocs and (i % self.ISplitByDocs == 0): # if self.ISplitByDocs == 0 then everything goes into one file; if this > 0 then
                    SPartFile = "part" + str(1000000 + i) + self.SOutput_file # generate new file name
                    FOut.flush()
                    FOut.close()
                    FOut = open(SPartFile, 'w')
        FOut.flush()
        FOut.close()

        return


    def procFile(self, SIn,SFNameIn,i): # sending each json string for extraction of text and attaching an correct tags to each output string output string
        STagOpen = '<' + self.STag + ' id="' + self.STag + str(self.ID + i)  + '">\n'
        STagClose = '\n</' + self.STag + '>\n\n'
        SText4Corpus = self.getJson(SIn, SFNameIn)
        if SText4Corpus:
            return STagOpen + SText4Corpus + STagClose
        else:
            print('\tNo data read from: ' + SFNameIn)
            return None


    def getJson(self, SIn, SFNameIn): # for each file-level string read from a file: managing internal structure of the covid-19 json file
        LOut = [] # collecting a list of strings
        try:
            DDoc = json.loads(SIn)
        except:
            print('\t\t' + SFNameIn + ' => error reading json2dictionary')
            return None
        # metadata:
        try:
            DMetaData = DDoc['metadata']
            if DMetaData:
                SMetaData = self.getJson_Metadata(DMetaData)
                if SMetaData: LOut.append(SMetaData)
        except:
            print('\t\t\t' + SFNameIn + ' ====> no metadata')
            DMetaData = None
        # body text
        try:
            LBodyText = DDoc['body_text']
            if LBodyText:
                SBodyText = self.getJson_BodyText(LBodyText)
                LOut.append(SBodyText)
        except:
            print('\t\t\t' + SFNameIn + ' ====> no body_text')
            LBodyText = None
        # further: to implement references

        SText = '\n\n'.join(LOut)
        return SText


    def getJson_Metadata(self, DIn): # converts interesting parts of metadata into a string
        SMetadata = ''
        LMetadata = []
        try: STitle = DIn["title"]
        except: STitle = None
        if STitle and self.BInclTitle:
            LMetadata.append(STitle)

        # to implement reading of authors' names

        if LMetadata: SMetadata = '\n\n'.join(LMetadata)
        return SMetadata


    def getJson_BodyText(self, LIn): # converts interesting parts of the body texts into a string
        SBodyText = ''
        LBodyText = []
        for DParagraph in LIn:
            try:
                ## DParagraphs[section] ## -- later on >> distinction between different sections....
                SParagraph = DParagraph["text"]
                LBodyText.append(SParagraph)
            except:
                print('!',)
                continue

        SBodyText = '\n\n'.join(LBodyText)
        return SBodyText


'''
if __name__ == '__main__':
    OJsonDir2txt = clJsonDir2txt(sys.argv[1], output_file = 'covid19corpus.txt', include_title = True, include_refs = False, split_by_docs=0)
'''
    # arguments:
    '''
        sys.argv[1], # obligatory: input directory name;
            other arguments optional:
            output_file = 'covid19corpus.txt',
            include_title = True, # include or exclude title
            include_refs = False, # not implemented yet: include or exclude references
            split_by_docs=0 # split by groups of n documents; if 0 then write to one file
    '''
